# Prequisites

In [43]:
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from boxmot import BoostTrack
from boxmot import StrongSort
import numpy as np
import torch

from collections import defaultdict
import os
from pathlib import Path

import torchvision
from torchvision.transforms import Compose, Lambda, Normalize, CenterCrop, Resize, ToTensor
from torchvision.transforms._transforms_video import CenterCropVideo, NormalizeVideo
import cv2

# from pytorchvideo.transforms import ShortSideScale

# Detector

In [2]:
detector_model = YOLO("yolo11s.pt")

In [3]:
def object_detection(img):
    lis = []
    results = detector_model.predict(img, verbose=False, device="cuda")
    temp = img.copy()
    for result in results:
        pass
    length = len(result.boxes.data)
    for index in range(length):
        boxes = result.boxes
        xyxy = boxes.xyxy[index].detach().cpu()
        x1, y1, x2, y2 = int(xyxy[0].item()), int(xyxy[1].item()), int(xyxy[2].item()), int(xyxy[3].item())
        # xywh = boxes.xywh[0].detach().cpu()
        # x1, y1, w, h = int(xywh[0].item()), int(xywh[1].item()), int(xywh[2].item()), int(xywh[3].item())
        cls_id = int(result.boxes.cls[index].detach().cpu().item())
        if(cls_id != 0): #Only person detection trying
            continue
        confidence = float(result.boxes.conf[index].detach().cpu().item())
        cv2.rectangle(temp, (x1, y1), (x2, y2), (255,0,0), 2)
        cv2.putText(temp, text="Person", org=(x1, y1-5), color=(255,0,0), fontScale=0.2, fontFace=cv2.FONT_HERSHEY_DUPLEX)
        res = [x1, y1, x2, y2, confidence, cls_id]
        lis.append(res)
    return lis


# Tracker

In [4]:
if torch.cuda.is_available():
    # device = torch.device("cuda")
    device = "cuda"
else:
    device = torch.device("cpu")

#tracker = BoostTrack(reid_weights='osnet_x0_25_msmt17.pt', device=device, half=False)

tracker = StrongSort(reid_weights=Path("osnet_x0_25_market1501.pt"), device="cpu", 
                     half=False, cmc=False) #CMC is motion compensation

2025-10-05 20:31:04.005 | INFO     | boxmot.trackers.basetracker:__init__:56 - BaseTracker initialization parameters:
2025-10-05 20:31:04.005 | INFO     | boxmot.trackers.basetracker:__init__:57 - det_thresh: 0.3
2025-10-05 20:31:04.006 | INFO     | boxmot.trackers.basetracker:__init__:58 - max_age: 30
2025-10-05 20:31:04.006 | INFO     | boxmot.trackers.basetracker:__init__:59 - max_obs: 50
2025-10-05 20:31:04.006 | INFO     | boxmot.trackers.basetracker:__init__:60 - min_hits: 3
2025-10-05 20:31:04.007 | INFO     | boxmot.trackers.basetracker:__init__:61 - iou_threshold: 0.3
2025-10-05 20:31:04.008 | INFO     | boxmot.trackers.basetracker:__init__:62 - per_class: False
2025-10-05 20:31:04.008 | INFO     | boxmot.trackers.basetracker:__init__:63 - nr_classes: 80
2025-10-05 20:31:04.008 | INFO     | boxmot.trackers.basetracker:__init__:64 - asso_func: iou
2025-10-05 20:31:04.009 | INFO     | boxmot.trackers.basetracker:__init__:65 - is_obb: False
2025-10-05 20:31:04.009 | INFO     | bo

In [5]:
def visualize_tracking(tracker_results, img):
    for i in tracker_results:
        x1, y1, x2, y2 = int(i[0]), int(i[1]), int(i[2]), int(i[3])
        track_id = int(i[4])
        track_confidence = float(i[5])
        cv2.rectangle(img, (x1,y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(img, text=str(track_id), org=(x1, y1-5), color=(255,0,0), fontScale=0.2, fontFace=cv2.FONT_HERSHEY_DUPLEX)
    return img 

def save_video_clip(track__buffers, out_path, track_id):
    h, w, _ = track__buffers[track_id][0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(out_path, fourcc, 15, (w, h))
    for f in track__buffers[track_id]:
        out.write(cv2.cvtColor(f, cv2.COLOR_BGR2RGB))
    out.release()

# Resnet 3D

In [49]:
video_model = torchvision.models.video.r3d_18(weights=torchvision.models.video.R3D_18_Weights)

c:\Users\Snowwolf\miniconda3\envs\slowfast\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [50]:
transform = Compose([
    Lambda(lambda x: x / 255.0),
    CenterCropVideo(256),
    NormalizeVideo((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
])

In [51]:
def run_action_recognition(track_buffer):
    clip = np.stack(track_buffer) # (T, H, W, 3) -> (32, H, W, 3)
    clip = torch.from_numpy(clip).permute(3,0,1,2) # [C, T, H, W]

    clip = transform(clip)

    inputs = clip.unsqueeze(0).float() #Adding batch dimension

    with torch.no_grad():
        preds = video_model(inputs)

    return preds

In [52]:
weights = torchvision.models.video.R3D_18_Weights.DEFAULT
categories = weights.meta["categories"]

## With Tracker

In [57]:
path = "video1.mp4"
cap = cv2.VideoCapture(path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

output_path = "Resnet3D-wt-4.mp4"

h, w = frame_height, frame_width 
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# Initialize tqdm progress bar

In [58]:
crop_size = 256
track__buffers = defaultdict(list)
save_dir = "clips"
frame_idx = 0
pred_class = "-"
pbar = tqdm(total=total_frames, desc="Processing video", unit="frame")

while True:
    success, frame = cap.read()
    if not success:
        print("Video ended")
        break 
    frame_idx += 1
    pbar.update(1)
    # if(frame_idx % 3 != 0):
    #     continue
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    copy_frame = frame.copy()
    detections = np.array(object_detection(frame))
    if(detections.size == 0):
        out.write(frame)
        continue
    tracker_results = tracker.update(detections, frame)
    #print(tracker_results)
    #annotated_img = visualize_tracking(tracker_results, frame)
    #Cropping separately as per id
    if(tracker_results.size == 0):
        out.write(frame)
        continue
    for x1,y1,x2,y2, track_id, conf, cls_id, frame_id in tracker_results:
        if(x2-x1 < 10 and y2-y1 < 10):
            out.write(frame)
            continue
        x1,y1,x2,y2 = int(x1), int(y1), int(x2), int(y2)
        crop = copy_frame[y1:y2, x1:x2]
        if crop.size == 0 or crop.shape[0] == 0 or crop.shape[1] == 0: #For crop being empty or invalid
            out.write(frame)
            continue
        crop = cv2.resize(crop, (crop_size, crop_size))
        track__buffers[track_id].append(crop)
        cv2.rectangle(frame, (x1,y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, text=str(track_id), org=(x1, y1-5), color=(255,0,0), fontScale=1, fontFace=cv2.FONT_HERSHEY_COMPLEX)
        cv2.putText(frame, text=str(pred_class), org=(x1+15, y1-5), color=(255,0,0), fontScale=2, fontFace=cv2.FONT_HERSHEY_COMPLEX)

        if len(track__buffers[track_id]) == 32:
            out_path = os.path.join(save_dir, f"id_{int(track_id)}_frame_{int(frame_idx)}.mp4")
            #save_video_clip(track__buffers, out_path, track_id)
            preds = run_action_recognition(track__buffers[track_id])
            probs = torch.nn.functional.softmax(preds, dim=1)
            top_prob, top_class = torch.max(probs, dim=1)
            pred_class = categories[top_class]
            prob = top_prob.item()
            track__buffers[track_id] = []
    out.write(frame)
    # cv2.imshow("Tracker_frame", frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break


cap.release()
cv2.destroyAllWindows()
out.release()

Processing video:   0%|          | 0/300 [00:00<?, ?frame/s]

Video ended


In [99]:
crop

array([], shape=(85, 0, 3), dtype=uint8)

# Without Tracker

In [59]:
path = "video1.mp4"
cap = cv2.VideoCapture(path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

output_path = "Resnet3D-4.mp4"

h, w = frame_height, frame_width 
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [60]:
save_dir = "clips"
frame_idx = 0
pred_class = "-"

pbar = tqdm(total=total_frames, desc="Processing video", unit="frame")
while True:
    success, frame = cap.read()
    if not success:
        print("Video ended")
        break 
    frame_idx += 1
    pbar.update(1)
    # if(frame_idx % 3 != 0):
    #     continue
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    cv2.putText(frame, text=str(pred_class), org=(40, 40), color=(0,0,255), fontScale=2, fontFace=cv2.FONT_HERSHEY_COMPLEX)
    out.write(frame)
    img = cv2.resize(frame, (256, 256))
    track__buffers[0].append(img)
    if len(track__buffers[0]) == 32:
        preds = run_action_recognition(track__buffers[0])
        probs = torch.nn.functional.softmax(preds, dim=1)
        top_prob, top_class = torch.max(probs, dim=1)
        pred_class = categories[top_class]
        cv2.putText(frame, text=str(pred_class), org=(40, 40), color=(0,0,255), fontScale=2, fontFace=cv2.FONT_HERSHEY_COMPLEX)
        prob = top_prob.item()
        out.write(frame)
        track__buffers[0] = []
    # cv2.imshow("Tracker_frame", frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break


cap.release()
cv2.destroyAllWindows()
out.release()

Processing video:   0%|          | 0/300 [00:00<?, ?frame/s]

Video ended
